In [3]:
import os
import numpy as np

In [7]:
data = {}
data['angry'] = np.array([])
data['fear'] = np.array([])
data['happy'] = np.array([])
data['sad'] = np.array([])
for folder in os.listdir('data'):
    for file in os.listdir('data/{folder}'):
        data[folder] = np.append(data[folder], 'data/{folder}/{file}')

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'data/{folder}'

In [8]:
data

{'angry': array([], dtype=float64),
 'fear': array([], dtype=float64),
 'happy': array([], dtype=float64),
 'sad': array([], dtype=float64)}

Step 1: Split the dataset into training and testing sets